In [ ]:
# the first two cells must be executed

import sys
sys.path.insert(0, "..")

In [ ]:
%matplotlib notebook

# FlexRiLoG - constructing flexible realizations via edge colorings

Jan Legerský

 *Department of Applied Mathematics, Faculty of Information Technology, Czech Technical University in Prague*
 
Special Semester on Rigidity and Flexibility, workshop Code of Rigidity
 
Jupyter notebook: https://jan.legersky.cz/CodeOfRigidity

### SageMath

* free and open-source computer algebra system
* started by William Stein in 2005
* syntax resembling Python
* instead reinventing wheel, SageMath integrates existing software packages, for instance:
  - Algebraic geometry: Singular
  - Linear algebra: ATLAS, BLAS, LAPACK, NumPy
  - Graph theory: NetworkX


### Algebraic Geometry: Circuit polynomial of wheel with 5 vertices

In [ ]:
R.<l01, l02, l03, l04, l12, l13, l14, l23, l24, l34> = PolynomialRing(QQ, order='lex')

In [ ]:
CMupper = matrix([[0, 1, 1, 1, 1, 1],
                  [0, 0, l01, l02, l03, l04],
                  [0, 0, 0, l12, l13, l14],
                  [0, 0, 0, 0, l23, l24],
                  [0, 0, 0, 0, 0, l34,],
                  [0, 0, 0, 0, 0, 0]])
CM = CMupper + CMupper.transpose()
show(CM)

In [ ]:
I = ideal(CM.minors(5)); I

In [ ]:
graphs.WheelGraph(5)  # graphs provides many predefined graphs

In [ ]:
basis = I.elimination_ideal([l13,l24]).basis
len(basis)

In [ ]:
basis

In [ ]:
circuit_pol = basis[0]
len(circuit_pol.monomials())

### FlexRiLoG

* **Flex**ible and **Ri**gid **L**abelings **o**f **G**raphs
* available at https://github.com/Legersky/flexrilog
* for research experiments
* object-oriented, the main class FlexRiGraph inherited from Graph provided by SageMath

### Flexible Frameworks

A *realization* of a graph $G=(V_G, E_G)$ is a map
$$p:V_G \rightarrow \mathbb{R}^2$$
such that $p(u)\neq p(v)$ if $uv \in E_G$.

A *framework* $(G,p)$ is *flexible* if there is a continuous curve  starting at $p$ of non-congruent realizations that have the same edge lengths as $(G,p)$.

### Dixon I flex of $K_{3,3}$

In [ ]:
from flexrilog import FlexRiGraph, GraphMotion
t = var('t')
K33 = FlexRiGraph(graphs.CompleteBipartiteGraph(3,3))
parametrization = {0: vector([-sqrt(2+sin(t)^2),0]),    1: vector([sin(t),0]),
                   2: vector([sqrt(1+sin(t)^2),0]),    3: vector([0,sqrt(1+cos(t)*cos(t))]),
                   4: vector([0,-sqrt(2+cos(t)^2)]),    5: vector([0,cos(t)]),}
motion_K33 = GraphMotion.ParametricMotion(K33,parametrization,'symbolic',check=True)

In [ ]:
show(parametrization[2],parametrization[3])

In [ ]:
motion_K33.animation_SVG(edge_partition=False)

### NAC-colorings

A coloring of edges $\delta : E_G \rightarrow \{blue, red\}$ is called a *NAC-coloring*,
if it is surjective and for every cycle in G , either all edges in the
cycle have the same color, or there are at least two blue and two
red edges in the cycle.

In [ ]:
C4 = FlexRiGraph(graphs.CycleGraph(4))
C4.show_all_NAC_colorings()

### Theorem (Grasegger, L., Schicho, 2019)
A connected graph with at least one edge has a flexible realization if and only if it has a NAC-coloring.

In [ ]:
from flexrilog import GraphDrawer
drawer = GraphDrawer()

In [ ]:
G = drawer.get_graph(); G

In [ ]:
G.

In [ ]:
print(G.NAC_colorings())
delta = G.NAC_colorings()[0]

In [ ]:
delta
GraphMotion.GridConstruction(G,delta,zigzag=False).animation_SVG(edge_partition=delta)

In [ ]:
G.has_injective_grid_construction()

### Rotationally symetric TP-frameworks

In [ ]:
from flexrilog import Pframework, CnSymmetricFlexRiGraphCartesianNACs
from flexrilog import CnSymmetricFlexRiGraph, GraphGenerator

In [ ]:
P = GraphGenerator.PenroseFramework(6,numeric=True,radius=10)
plot_args = {'vertex_labels':False,'vertex_size':0, 'edge_thickness':1}
Pplot = P.plot(**plot_args)
filling = point2d([])
for a,b,c,d in P.four_cycles():
    if abs(RR((vector(P._pos[a])-vector(P._pos[b]))*(vector(P._pos[c])-vector(P._pos[b])))) < 0.4:
        filling += polygon([P._pos[v] for v in [a,b,c,d]], color='lightblue', axes=False)
Pplot + filling

In [ ]:
def findPentaStars(P):
    res = []
    for v in P.vertices(sort=False):
        if P.degree(v)==5:
            if sum([1 for u in P.neighbors(v) if P.degree(u)==3])==5:
                res.append([v, [u for u in P.neighbors(v)], P.distance('0',v)])
    return res
braces = [[u,v]  for S in findPentaStars(P) for u,v in Subsets(S[1],2) 
          if ((vector(P._pos[u])-vector(P._pos[v])).norm() < 1.5 and S[2] in [6, 9])]
Pbraced = FlexRiGraph(P.edges(sort=False)+braces,pos=P._pos,check=False)
Pbraced.plot(**plot_args)+filling

In [ ]:
sym = CnSymmetricFlexRiGraphCartesianNACs.Cn_symmetries_gens(Pbraced,5)
PenroseBraced = CnSymmetricFlexRiGraphCartesianNACs(Pbraced, sym)

In [ ]:
PenroseBraced.Cn_symmetric_NAC_colorings()

A NAC-coloring is *Cartesian* if no two vertices are connected by a red and blue path simultaneously.

In [ ]:
FlexRiGraph(graphs.CycleGraph(4)).show_all_NAC_colorings()

In [ ]:
deltaP = PenroseBraced.Cn_symmetric_NAC_colorings()[-1]
deltaP.plot(**plot_args)

In [ ]:
M = Pframework(PenroseBraced,PenroseBraced._pos,check=False).flex_from_cartesian_NAC(deltaP)
M.animation_SVG(edge_partition=deltaP,vertex_labels=False,totalTime=24)

### Theorem (Grasegger, L., 2024)
A rotationally symmetric TP-framework is flexible if and only if the graph has a rotationally symmetric Cartesian NAC-coloring.

### Reflection symmetry

In [ ]:
from flexrilog import CsSymmetricFlexRiGraph
Gcs = FlexRiGraph([(0, 1), (0, 2), (0, 3), (0, 9), (1, 2), (1, 7), (1, 8), (2, 4), (2, 6),
                   (3, 4), (3, 8), (4, 5), (4, 8), (5, 6), (6, 7), (6, 9), (7, 9)])
Cs_sym = CsSymmetricFlexRiGraph.Cs_symmetries_gens_according_isomorphic_orbits(Gcs)[0]
G_Cs = CsSymmetricFlexRiGraph(Gcs,Cs_sym)
G_Cs.set_symmetric_positions({ 1: [1, 0], 2: [0, 1.3], 5: [0, 3.5], 
                              6: [1.4, 2.5 ], 7: [2, 0.9], 9: [1.1, 1.5]})
G_Cs

### Theorem (Dewar, Grasegger, L., 2024+)
If a reflection-symmetric framework with symmetry $\sigma$ is flexible, then the graph has a *pseudo-RS-coloring*, which is an edge colouring
$\delta:E_G\rightarrow \{red,blue,gold\}$ such that:
 * $ \{ red,blue \}\subseteq \delta(E_G) \subseteq \{ red,blue, gold \}$,
 * changing gold to blue results in a NAC-colouring,
 * changing gold to red results in a NAC-colouring,
 * $\delta(e) = red$ if and only if $\delta(\sigma e) = blue$ for all $e\in E_G$,  and
 * if $\delta(e) = gold$ then $\delta(\sigma e) = gold$ for all $e\in E_G$.

In [ ]:
G_Cs.show_all_pseudoRScolorings()

In [ ]:
from flexrilog import colB, colG, colR
d_Cs = G_Cs.pseudoRScolorings()[1]
GraphMotion.CsSymmetricGridConstruction(G_Cs, d_Cs).animation_SVG(colors=[colR,colB,colG],
                edge_partition=[d_Cs.red_edges(),d_Cs.blue_edges(),d_Cs.golden_edges()])

In [ ]:
C5 = CsSymmetricFlexRiGraph(graphs.CycleGraph(5),PermutationGroupElement([(2,3),(1,4)]))
C5.show_all_pseudoRScolorings()

In [ ]:
delta5 = C5.pseudoRScolorings()[0]
GraphMotion.CsSymmetricGridConstruction(C5, delta5).animation_SVG(colors=[colR,colB,colG],
                edge_partition=[delta5.red_edges(),delta5.blue_edges(),delta5.golden_edges()])

### Thank you

![Penrose](penrose.svg)